In [97]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

df = pd.read_csv("./data/Planning_Combined.csv")
del df["Product"]
del df["FulfilPerc"]
del df["Delivery Week"]
df.head()

,Producer Code,Product ID,Quantity,Cost,Fulfilled,Water Access,Land Ownership
0,PBY,3166,10,5000,0,False,False
1,SGC,3188,25,2465,25,NaN,NaN
2,LXF,3613,18,2631,18,NaN,NaN
3,HAF,3755,12,3162,0,False,True
4,LXF,3132,60,3162,60,NaN,NaN


In [98]:
df["Producer Code"].astype('category')
df["Product ID"].astype('category')
df["Water Access"].astype('category')
df["Land Ownership"].astype('category')


0       False
1         NaN
2         NaN
3        True
4         NaN
        ...  
1587     True
1588     True
1589     True
1590    False
1591    False
Name: Land Ownership, Length: 1592, dtype: category
Categories (2, bool): [False, True]

In [99]:
Y = df["Fulfilled"]
X = df[["Quantity","Cost"]]
X = sm.add_constant(X)
ks = sm.OLS(Y, X)
ks_res =ks.fit()
ks_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Fulfilled   R-squared:                       0.218
Model:                            OLS   Adj. R-squared:                  0.217
Method:                 Least Squares   F-statistic:                     220.9
Date:                Sat, 04 Feb 2023   Prob (F-statistic):           2.27e-85
Time:                        06:21:04   Log-Likelihood:                -5832.4
No. Observations:                1592   AIC:                         1.167e+04
Df Residuals:                    1589   BIC:                         1.169e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8291      1.102     -0.752      0.452      -2.991       1.333
Quantity       0.4257      0.020     20.833      0.000       0.386       0.466
Cost           0.0005      0.000      1.948      0.052   -3.53e-06       0.001
==============================================================================
Omnibus:                      106.294   Durbin-Watson:                   1.817
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              307.686
Skew:                           0.325   Prob(JB):                     1.54e-67
Kurtosis:                       5.053   Cond. No.                     1.76e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.76e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [100]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder(handle_unknown="use_encoded_value",unknown_value=-1, encoded_missing_value=-1)
df = df[["Producer Code","Product ID","Water Access","Land Ownership","Quantity","Cost"]]
df[["Producer Code","Product ID","Water Access","Land Ownership"]] = enc.fit_transform(df[["Producer Code","Product ID","Water Access","Land Ownership"]]).astype(int)
df.head()

,Producer Code,Product ID,Water Access,Land Ownership,Quantity,Cost
0,37,7,0,0,10,5000
1,43,12,-1,-1,25,2465
2,27,33,-1,-1,18,2631
3,14,39,0,1,12,3162
4,27,4,-1,-1,60,3162


In [101]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
Xfull = df[["Producer Code","Product ID","Water Access","Land Ownership","Quantity","Cost"]]
train_X, val_X, train_y, val_y = train_test_split(Xfull, Y, random_state = 0)
model = RandomForestRegressor(random_state=0)
model.fit(train_X, train_y)
preds_val = model.predict(val_X)
mae = mean_absolute_error(val_y, preds_val)
mae

5.7240649327023885

In [107]:
def predictor(prod_code, prod_id, water, land,quantity, cost):
    data = np.append(enc.transform([[prod_code,prod_id,water,land]])[0],[quantity,cost])
    print(model.predict([data]))

predictor("PBY",3166,0,0,10,5000)

[0.0125]


c:\Users\Kanis\goodaly\packages\backend\venv\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but OrdinalEncoder was fitted with feature names
  warnings.warn(
c:\Users\Kanis\goodaly\packages\backend\venv\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
